In [212]:
# 載入套件
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time

In [1]:
# 載入爬蟲相關套件
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# 因為使用 Chrome 瀏覽器當模擬器，因此將 Chorme 的驅動程式與 ipynb 放在同樣路徑下
from selenium.webdriver.chrome.service import Service
service = Service(executable_path = "chromedriver")

In [61]:
driver = webdriver.Chrome()

In [62]:
# 打開網頁
driver.get("https://structurednotes-announce.tdcc.com.tw/Snoteanc/")

In [65]:
# 透過 BeautifulSoup 解析網站的 html
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
print(soup)

<html><head>
<title>境外結構型商品資訊觀測站(資訊公告平台)
</title>
</head>
<frameset border="0" cols="*" frameborder="NO" framespacing="0" rows="85,*">
<frame name="topFrame" noresize="" scrolling="NO" src="/Snoteanc/include/b-top.htm"/>
<frameset border="0" cols="217,*" frameborder="NO" framespacing="0">
<frame name="Menu" scrolling="Y" src="/Snoteanc/include/Menu.jsp"/>
<frame name="mainFrame" scrolling="Y" src="/Snoteanc/include/Main.jsp"/>
</frameset>
</frameset>
<noframes><body>
</body></noframes>
</html>


In [71]:
# 需要轉換到要點的 frame 裡的 html
driver.switch_to.frame("Menu")

In [72]:
# 用絕對路徑找出點擊的圖示
from selenium.webdriver.common.by import By
element = driver.find_element(By.XPATH, "/html/body/table/tbody/tr[4]/td/a/img")

In [73]:
element.click()

In [79]:
driver.switch_to.default_content()

In [81]:
source_2 = driver.page_source
soup_2 = BeautifulSoup(source_2, 'html.parser')
print(soup_2)

<html><head>
<title>境外結構型商品資訊觀測站(資訊公告平台)
</title>
</head>
<frameset border="0" cols="*" frameborder="NO" framespacing="0" rows="85,*">
<frame name="topFrame" noresize="" scrolling="NO" src="/Snoteanc/include/b-top.htm"/>
<frameset border="0" cols="217,*" frameborder="NO" framespacing="0">
<frame cd_frame_id_="88262801d4bb2453bd99b5a407b0ca8d" name="Menu" scrolling="Y" src="/Snoteanc/include/Menu.jsp"/>
<frame name="mainFrame" scrolling="Y" src="/Snoteanc/include/Main.jsp"/>
</frameset>
</frameset>
<noframes><body>
</body></noframes>
</html>


In [83]:
driver.switch_to.frame("mainFrame")

In [88]:
# 找出下拉式選單的原件
select_institution = Select(driver.find_element(By.XPATH,
                                        "/html/body/form/table/tbody/tr[5]/td[2]/table/tbody/tr[3]/td[2]/div/select"))

In [90]:
# 選擇星展銀行
select_institution.select_by_value("50d2d3aa:13e906cdbb7:-769a")

In [93]:
# 點擊搜尋
element_2 = driver.find_element(By.XPATH, "/html/body/form/table/tbody/tr[5]/td[2]/table/tbody/tr[10]/td/input")
element_2.click()

In [115]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [131]:
# 找到需要爬蟲的表格
tables = soup.find_all('table')
second_table = tables[2] 
second_table

<table border="1" bordercolor="#D9B55C" cellpadding="5" cellspacing="0" width="100%">
<tbody><tr>
<td align="center" class="unnamed1" onclick="reSortResultList('FUND_CODE','0')" rowspan="2"><strong><font color="blue" size="2"><u>商品代號</u></font></strong>
<img src="/Snoteanc/images/arrowDown.jpg"/>
</td>
<td align="center" class="unnamed1" onclick="reSortResultList('FUND_NAME','0')" rowspan="2"><strong><font color="blue" size="2"><u>商品名稱</u></font></strong> </td>
<td align="center" class="unnamed1" colspan="6"><strong><font size="2">基本資料</font></strong></td>
<td align="center" class="unnamed1" colspan="4"><strong><font size="2">相關機構</font></strong></td>
</tr>
<tr>
<td align="center" class="unnamed1" onclick="reSortResultList('FUND_CURR','0')"><strong><font color="blue" size="2"><u>計價幣別</u></font></strong> </td>
<td align="center" class="unnamed1" onclick="reSortResultList('FUND_STOP_DATE','0')"><strong><font color="blue" size="2"><u>終止<br/>
					生效日</u></font></strong> </td>
<td align="c

In [166]:
# 找到表格行
rows = second_table.find_all('tr')

# 初始化一个空的列表來儲存數據
data = []

# 用迴圈抓出表格內容
for row in rows:
    # 找到表格數據單元格
    cells = row.find_all('td')
    
    # 提取每个單元的文本内容并添加到數據列表
    row_data = [cell.get_text(strip=True) for cell in cells]
    data.append(row_data)

data

['037001003122',
 'SG ISSUER 美元計價12個月固定配息（記憶式自動提前出場）股權連結結構型商品（不保本及無擔保）',
 '美元',
 '2018/06/19',
 '股權',
 '0%',
 '專業投資人',
 '2018/02/14',
 'SG ISSUER',
 '法商法國興業銀行台北分行',
 '星展(台灣)商業銀行股份有限公司']

In [167]:
# 列表前兩個儲存的是所需要的 coulmns
data[0], data[1]

(['商品代號', '商品名稱', '基本資料', '相關機構'],
 ['計價幣別',
  '終止生效日',
  '連結標的類別',
  '計價貨幣本金保本率',
  '投資人類別',
  '商品國內發行日',
  '發行機構',
  '總代理',
  '受託或銷售機構'])

In [168]:
import pandas as pd
# 刪除 "基本資料", "相關機構"
data[0] = [col for col in data[0] if col not in ["基本資料", "相關機構"]]
data[0]

['商品代號', '商品名稱']

In [169]:
column_names = data[0] + data[1]
# list 後面儲存的就是每一行的資料
data = data[2:]
# 将數據轉換成 dataframe
df = pd.DataFrame(data, columns = column_names)

In [170]:
df.head()

,商品代號,商品名稱,計價幣別,終止生效日,連結標的類別,計價貨幣本金保本率,投資人類別,商品國內發行日,發行機構,總代理,受託或銷售機構
0,037001000216,SG ISSUER 10年期美元計價( 發行機構可買回 )連結雙標的每日計息結構型商品(無擔保),美元,2023/09/10,"匯率, 商品",100%,專業投資人,2013/09/10,SG ISSUER,法商法國興業銀行台北分行,星展(台灣)商業銀行股份有限公司
1,037001002827,SG ISSUER 美元計價12個月固定配息（記憶式自動提前出場）股權連結結構型商品（不保本...,美元,2018/02/01,股權,0%,專業投資人,2017/12/27,SG ISSUER,法商法國興業銀行台北分行,星展(台灣)商業銀行股份有限公司
2,037001002837,SG ISSUER 美元計價12個月固定配息（記憶式自動提前出場）股權連結結構型商品（不保本...,美元,2018/02/01,股權,0%,專業投資人,2017/12/28,SG ISSUER,法商法國興業銀行台北分行,星展(台灣)商業銀行股份有限公司
3,037001002838,SG ISSUER 澳幣計價12個月固定配息（記憶式自動提前出場）股權連結結構型商品（不保本...,澳幣,2018/02/01,股權,0%,專業投資人,2017/12/28,SG ISSUER,法商法國興業銀行台北分行,星展(台灣)商業銀行股份有限公司
4,037001002840,SG ISSUER 歐元計價12個月固定配息（記憶式自動提前出場）股權連結結構型商品（不保本...,歐元,2019/01/03,股權,0%,專業投資人,2017/12/28,SG ISSUER,法商法國興業銀行台北分行,星展(台灣)商業銀行股份有限公司


In [211]:
# 轉換成 UTF-8 並儲存為 CSV
df.to_csv("submission.csv", encoding = 'utf_8_sig', index = False)